In [ ]:
# 初回のみ
#!wget -P '/content/drive/My Drive/' http://mdcrosby.com/builds/AnimalAI_LINUX_3.0.zip
#!unzip '/content/drive/MyDrive/AnimalAI_LINUX_3.0.zip' -d "/content/drive/MyDrive/最終課題/animal-ai/"
#!mv /content/drive/MyDrive/最終課題/animal-ai/AnimalAI_LINUX_3.0/* /content/drive/MyDrive/最終課題/animal-ai/env/

In [1]:
!pip install -e animalai
!pip install stable_baselines3 
!pip install gym_unity
!chmod +x /content/drive/MyDrive/animal-ai/env/UnityPlayer.so
!chmod +x /content/drive/MyDrive/animal-ai/env/AnimalAI.x86_64
!apt update
!apt install xvfb

Obtaining file:///content/drive/MyDrive/%E6%9C%80%E7%B5%82%E8%AA%B2%E9%A1%8C/animal-ai/animalai
  Using cached mlagents_envs-0.27.0-py3-none-any.whl (75 kB)
  Attempting uninstall: mlagents-envs
    Found existing installation: mlagents-envs 0.28.0
    Uninstalling mlagents-envs-0.28.0:
      Successfully uninstalled mlagents-envs-0.28.0
  Attempting uninstall: animalai
    Found existing installation: animalai 3.0.0
    Can't uninstall 'animalai'. No files were found to uninstall.
  Running setup.py develop for animalai
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym-unity 0.28.0 requires mlagents-envs==0.28.0, but you have mlagents-envs 0.27.0 which is incompatible.
     |████████████████████████████████| 1.6 MB 8.7 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.19.0-py3-none-any.whl size=1663117 sha256=6cf27e04f882230cb1875d657a157e68ce490a170e

In [2]:
from stable_baselines3 import DQN
import torch as th

import sys
import random
import os
# from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from gym_unity.envs import UnityToGymWrapper
from animalai.animalai.envs.environment import AnimalAIEnvironment

In [ ]:
competition_folder = "./configs/competition/"
configuration_files = os.listdir(competition_folder)
configuration_random = random.randint(0, len(configuration_files))
configuration_file = (
    competition_folder + configuration_files[configuration_random]
)

aai_env = AnimalAIEnvironment(
        seed = 123,
        file_name="./env/AnimalAI",
        arenas_configurations=configuration_file,
        play=False,
        base_port=5000,
        inference=False,
        useCamera=True,
        resolution=36,
        useRayCasts=True,
        no_graphics=True,
        # raysPerSide=1,
        # rayMaxDegrees = 30,
    )

# env = UnityToGymWrapper(aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=False)
# def make_env():
#     def _thunk():
#         env = UnityToGymWrapper(aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True)
#         return env
#     return _thunk
# env = DummyVecEnv([make_env()])
env = UnityToGymWrapper(aai_env, uint8_visual=True, allow_multiple_obs=False, flatten_branched=True)
runname = "inserrunname"

policy_kwargs = dict(activation_fn=th.nn.ReLU)
model = DQN("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1, tensorboard_log=("./dqn_tensorboard/" + runname))

no_saves = 100
no_steps = 1000000
reset_num_timesteps = True
for i in range(no_saves):
    model.learn(no_steps, reset_num_timesteps=reset_num_timesteps)
    model.save("results/" + runname + "/model_" + str( (i+1)*no_steps ))
    reset_num_timesteps = False
env.close()